Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic model

In [5]:
batch_size = 128

regularization_strength = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Add regularization
    regularization = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    loss = tf.reduce_mean(loss + regularization_strength * regularization)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
train_subset = 10000

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.530151
Minibatch accuracy: 5.5%
Validation accuracy: 11.3%
Minibatch loss at step 500: 0.764190
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 1000: 0.713024
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 1500: 0.792104
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 2000: 0.764244
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 2500: 0.923355
Minibatch accuracy: 75.8%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.779274
Minibatch accuracy: 85.2%
Validation accuracy: 80.6%
Test accuracy: 87.3%


### Neural Network

In [7]:
hidden_layer = 1024
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {
        'h': tf.Variable(tf.random_normal([image_size * image_size, hidden_layer])),
        'out': tf.Variable(tf.random_normal([hidden_layer, num_labels]))
    }
    biases = {
        'h': tf.Variable(tf.zeros([hidden_layer])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }

    # Training computation.
    def get_network_out(train_set):
        # Hidden layer is relu
        hidden = tf.matmul(train_set, weights['h']) + biases['h']
        hidden = tf.nn.relu(hidden)

        # Out is an array with probabilities for every class
        return tf.matmul(hidden, weights['out']) + biases['out']

    train_out = get_network_out(tf_train_dataset)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_out))

    # Add regularization
    regularization = tf.nn.l2_loss(weights['h']) + tf.nn.l2_loss(biases['h'])
    regularization += tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(biases['out'])
    loss = tf.reduce_mean(loss + regularization_strength * regularization)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_out)
    valid_prediction = tf.nn.softmax(get_network_out(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_network_out(tf_test_dataset))

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4520.935059
Minibatch accuracy: 2.3%
Validation accuracy: 28.9%
Minibatch loss at step 500: 27.129904
Minibatch accuracy: 89.1%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.857695
Minibatch accuracy: 85.9%
Validation accuracy: 82.5%
Minibatch loss at step 1500: 0.689781
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.692625
Minibatch accuracy: 82.0%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 0.878364
Minibatch accuracy: 81.2%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.685061
Minibatch accuracy: 88.3%
Validation accuracy: 83.1%
Test accuracy: 89.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

train_dataset_small = train_dataset[:200]
train_labels_small = train_labels[:200]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4499.761230
Minibatch accuracy: 10.9%
Validation accuracy: 22.8%
Minibatch loss at step 500: 27.070856
Minibatch accuracy: 100.0%
Validation accuracy: 73.0%
Minibatch loss at step 1000: 0.383809
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 1500: 0.192735
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 2000: 0.186809
Minibatch accuracy: 100.0%
Validation accuracy: 75.0%
Minibatch loss at step 2500: 0.184363
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 3000: 0.186689
Minibatch accuracy: 100.0%
Validation accuracy: 74.7%
Test accuracy: 81.1%


As we can see on every batch of the training data the accuracy is 100% but on the validation set and on the test set the accuracy is lower. The model is so strong that is fits perfectly all the training cases and does not generalizes well. If we choose bigger reguarization strenth this might be solved

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
hidden_layer = 1024
graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {
        'h': tf.Variable(tf.random_normal([image_size * image_size, hidden_layer])),
        'out': tf.Variable(tf.random_normal([hidden_layer, num_labels]))
    }
    biases = {
        'h': tf.Variable(tf.zeros([hidden_layer])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    
    keep_probability = tf.placeholder(tf.float32)

    # Training computation.
    def get_network_out(train_set, add_dropout=False):
        # Hidden layer is relu
        hidden = tf.matmul(train_set, weights['h']) + biases['h']
        hidden = tf.nn.relu(hidden)
        if add_dropout:
            hidden = tf.nn.dropout(hidden, keep_probability)

        # Out is an array with probabilities for every class
        return tf.matmul(hidden, weights['out']) + biases['out']

    train_out = get_network_out(tf_train_dataset, add_dropout=True)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_out))

    # Add regularization
    regularization = tf.nn.l2_loss(weights['h']) + tf.nn.l2_loss(biases['h'])
    regularization += tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(biases['out'])
    loss = tf.reduce_mean(loss + regularization_strength * regularization)

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_out)
    valid_prediction = tf.nn.softmax(get_network_out(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_network_out(tf_test_dataset))

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_probability : 0.8}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4610.319824
Minibatch accuracy: 6.2%
Validation accuracy: 39.0%
Minibatch loss at step 500: 27.110205
Minibatch accuracy: 85.2%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.865750
Minibatch accuracy: 88.3%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 0.702222
Minibatch accuracy: 81.2%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.712222
Minibatch accuracy: 82.8%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.911004
Minibatch accuracy: 81.2%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.699200
Minibatch accuracy: 88.3%
Validation accuracy: 83.0%
Test accuracy: 89.4%


In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_probability : 0.4}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4820.689453
Minibatch accuracy: 7.0%
Validation accuracy: 22.7%
Minibatch loss at step 500: 27.812378
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 1000: 0.977213
Minibatch accuracy: 83.6%
Validation accuracy: 81.6%
Minibatch loss at step 1500: 0.789326
Minibatch accuracy: 78.1%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 0.770772
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 0.980706
Minibatch accuracy: 79.7%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.818029
Minibatch accuracy: 84.4%
Validation accuracy: 82.5%
Test accuracy: 89.0%


#### On the extremely overfitting model

In [13]:
num_steps = 3001

train_dataset_small = train_dataset[:200]
train_labels_small = train_labels[:200]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                keep_probability:0.2}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5103.678711
Minibatch accuracy: 10.2%
Validation accuracy: 29.7%
Minibatch loss at step 500: 28.123407
Minibatch accuracy: 100.0%
Validation accuracy: 74.8%
Minibatch loss at step 1000: 0.417796
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 0.219094
Minibatch accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 2000: 0.208294
Minibatch accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 2500: 0.215408
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 3000: 0.213721
Minibatch accuracy: 100.0%
Validation accuracy: 75.2%
Test accuracy: 81.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [16]:
hidden_layer_1 = 4096
hidden_layer_2 = 2048
hidden_layer_3 = 1024
hidden_layer_4 = 512
graph = tf.Graph()
regularization_strength = 0.001
import math
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = {
        'h1': tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_1],
                          stddev=math.sqrt(2.0/(image_size * image_size)))),
        'h2': tf.Variable(tf.truncated_normal([hidden_layer_1, hidden_layer_2],
                          stddev=math.sqrt(2.0/hidden_layer_1))),
        'h3': tf.Variable(tf.truncated_normal([hidden_layer_2, hidden_layer_3],
                          stddev=math.sqrt(2.0/hidden_layer_2))),
        'h4': tf.Variable(tf.truncated_normal([hidden_layer_3, hidden_layer_4],
                          stddev=math.sqrt(2.0/hidden_layer_3))),
        'out': tf.Variable(tf.truncated_normal([hidden_layer_4, num_labels],
                           stddev=math.sqrt(2.0/hidden_layer_4)))
    }
    biases = {
        'h1': tf.Variable(tf.zeros([hidden_layer_1])),
        'h2': tf.Variable(tf.zeros([hidden_layer_2])),
        'h3': tf.Variable(tf.zeros([hidden_layer_3])),
        'h4': tf.Variable(tf.zeros([hidden_layer_4])),
        'out': tf.Variable(tf.zeros([num_labels]))
    }
    
    keep_probability = tf.placeholder(tf.float32)
    
    def forward(current_in, layer, add_dropout=False):
        # Hidden layer is relu
        next_in = tf.matmul(current_in, weights[layer]) + biases[layer]
        next_in = tf.nn.relu(next_in)
        if add_dropout:
            next_in = tf.nn.dropout(next_in, keep_probability)
        
        return next_in

    # Training computation.
    def get_network_out(train_set, add_dropout=False):
        hidden = forward(train_set, 'h1', add_dropout=add_dropout)
        hidden = forward(hidden, 'h2', add_dropout=add_dropout)
        hidden = forward(hidden, 'h3', add_dropout=add_dropout)
        hidden = forward(hidden, 'h4', add_dropout=add_dropout)

        # Out is an array with probabilities for every class
        return tf.matmul(hidden, weights['out']) + biases['out']

    train_out = get_network_out(tf_train_dataset, add_dropout=True)

    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=train_out))

    # Add regularization
    weights_reg = [tf.nn.l2_loss(weigth) for weigth in weights.values()]
    biases_reg = [tf.nn.l2_loss(bias) for bias in biases.values()]
    regularization = sum(weights_reg) + sum(biases_reg)
    loss = tf.reduce_mean(loss + regularization_strength * regularization)

    # Optimizer.
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(train_out)
    valid_prediction = tf.nn.softmax(get_network_out(tf_valid_dataset))
    test_prediction = tf.nn.softmax(get_network_out(tf_test_dataset))

In [17]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data,
                 tf_train_labels : batch_labels,
                 keep_probability : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.053399
Minibatch accuracy: 14.8%
Validation accuracy: 11.3%


KeyboardInterrupt: 